# 二维卷积实验

## 0 数据加载

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import datasets, transforms
import os
from tqdm import tqdm
from PIL import Image

# --- 1. 自定义测试集 Dataset ---
class CustomTestDataset(Dataset):
    """用于加载无标签测试集（仅返回图像和文件名）"""

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [
            f
            for f in os.listdir(root_dir)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        try:
            self.image_files.sort(
                key=lambda x: int(x.split(".")[0].replace("image", ""))
            )
        except ValueError:
            self.image_files.sort()

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, img_name


# --- 2. 改进的数据加载函数（解决验证集Transform问题）---
def setup_data_loaders(
    data_root: str,
    image_size: int = 227,
    batch_size: int = 45,
    split_ratio: float = 0.8,
    num_workers: int = 4,
) -> tuple[DataLoader, DataLoader, DataLoader, int]:
    """
    设置和返回训练集、验证集和测试集的数据加载器，以及类别数量。
    
    关键改进：
    - 训练集使用数据增强
    - 验证集使用标准预处理（无增强）
    - 通过创建两个ImageFolder并用Subset索引的方式实现
    """

    # 2.1 配置路径
    TRAIN_DIR = os.path.join(data_root, "Train/Train")
    TEST_DIR = os.path.join(data_root, "Test/Test/Test1")

    # ImageNet 标准均值和标准差
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

    # 2.2 定义数据预处理
    # 训练集：包含数据增强
    train_transforms = transforms.Compose(
        [
            transforms.Resize((image_size, image_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),  # 添加轻微旋转
            transforms.ColorJitter(
                brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
            ),
            transforms.ToTensor(),
            transforms.Normalize(MEAN, STD),
        ]
    )

    # 验证集和测试集：仅标准预处理，无数据增强
    eval_transforms = transforms.Compose(
        [
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(MEAN, STD),
        ]
    )

    # 2.3 关键改进：创建两个ImageFolder实例
    # 一个用于训练（带增强），一个用于验证（不带增强）
    full_train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transforms)
    full_eval_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=eval_transforms)
    
    num_classes = len(full_train_dataset.classes)
    total_size = len(full_train_dataset)

    # 2.4 计算划分大小
    train_size = int(split_ratio * total_size)
    val_size = total_size - train_size

    # 2.5 生成索引列表（确保训练集和验证集不重叠）
    import random
    random.seed(42)  # 设置随机种子保证可复现
    indices = list(range(total_size))
    random.shuffle(indices)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]

    # 2.6 使用Subset创建数据集
    # 训练集使用带增强的transforms
    train_dataset = Subset(full_train_dataset, train_indices)
    # 验证集使用不带增强的transforms
    val_dataset = Subset(full_eval_dataset, val_indices)

    # 2.7 创建 DataLoader
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    # 2.8 加载测试集（无标签）
    test_dataset = CustomTestDataset(root_dir=TEST_DIR, transform=eval_transforms)

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    # 打印信息
    print(f"✓ 检测到的类别数量: {num_classes}")
    print(f"✓ 训练集大小: {len(train_dataset)} (使用数据增强)")
    print(f"✓ 验证集大小: {len(val_dataset)} (不使用数据增强)")
    print(f"✓ 测试集大小: {len(test_dataset)}")
    print(f"✓ 数据加载完成，Batch Size: {batch_size}")

    return train_loader, val_loader, test_loader, num_classes

## 1 手写二维卷积的实现，并在至少一个数据集上进行实验，从训练时间、预测精度、Loss变化等角度分析实验结果（最好使用图表展示）

In [ ]:
class Model(nn.Module):
    def __init__(
        self,
        chns_in: int,
        chns_out: int,
        ker_size: int,
        nums_classes: int,
        device: torch.device,
    ) -> None:
        super().__init__()
        self.chns_in = chns_in
        self.chns_out = chns_out
        self.ker_size = ker_size
        self.nums_classes = nums_classes
        self.conv = nn.Sequential(
            Model.MyConv2D(self.chns_in, self.chns_out, self.ker_size),
            nn.BatchNorm2d(self.chns_out),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 形状变为 (N, chns_out, 1, 1)
        self.fc = nn.Sequential(nn.Linear(self.chns_out, self.nums_classes))

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.conv(X)
        X = self.avgpool(X)  # 形状变为 (N, chns_out, 1, 1)
        X = X.flatten(1)  # 形状变为 (N, chns_out)
        X = self.fc(X)
        return X

    class MyConv2D(nn.Module):
        def __init__(self, chns_in: int, chns_out: int, ker_size: int) -> None:
            super().__init__()
            self.weight = nn.Parameter(
                torch.randn(size=(chns_out, chns_in, ker_size, ker_size))
            )
            self.bias = nn.Parameter(torch.randn(size=(1, chns_out, 1, 1)))

        def forward(self, X: torch.Tensor, padding=0, stride=1) -> torch.Tensor:
            return (
                Model.MyConv2D._corr2d_NCHW(X, self.weight, padding, stride)
                + self.bias  # (n,chns_out,(XH-KH+2P)/S,(XW-KW+2P)/S)
            )

        @staticmethod
        def _corr2d_HW(
            X_HW: torch.Tensor, K_HW: torch.Tensor, padding: int = 0, stride: int = 1
        ) -> torch.Tensor:
            # 扩展
            if padding > 0:
                X_HW = F.pad(X_HW, (padding, padding, padding, padding), "constant", 0)
            # 计算输出形状
            height_X, width_X = X_HW.shape
            height_K, width_K = K_HW.shape
            H_out = (height_X - height_K) // stride + 1
            W_out = (width_X - width_K) // stride + 1
            Y = torch.zeros((H_out, W_out), device=X_HW.device)
            # 滑动窗口循环
            for i in range(H_out):
                for j in range(W_out):
                    Y[i, j] = (
                        X_HW[
                            i * stride : i * stride + height_K,
                            j * stride : j * stride + width_K,
                        ]
                        * K_HW
                    ).sum()
            return Y

        @staticmethod
        def _corr2d_CHW(
            X_CHW: torch.Tensor, K_OIHW: torch.Tensor, padding: int = 0, stride: int = 1
        ) -> torch.Tensor:
            # 为循环做准备，定好循环次数
            chns_in = X_CHW.shape[0]
            chns_out = K_OIHW.shape[0]
            Y_list = []  # 用于保存每个输出通道的卷积结果
            for chn_out in range(chns_out):
                K_out = K_OIHW[chn_out]
                result = 0  # 用于保存当前输出通道的卷积结果(所有输入通道的卷积结果相加)
                for chn_in in range(chns_in):
                    Y = Model.MyConv2D._corr2d_HW(
                        X_CHW[chn_in], K_out[chn_in], padding, stride
                    )
                    result += Y
                Y_list.append(result)
            return torch.stack(Y_list, dim=0)

        @staticmethod
        def _corr2d_NCHW(
            X_NCHW: torch.Tensor, K_OIHW: torch.Tensor, padding: int, stride: int
        ) -> torch.Tensor:
            nums = X_NCHW.shape[0]
            Y_list = []
            for n in range(nums):
                Y = Model.MyConv2D._corr2d_CHW(X_NCHW[n], K_OIHW, padding, stride)
                Y_list.append(Y)
            return torch.stack(Y_list, dim=0)

In [10]:
class Loss:
    def __init__(self) -> None:
        self.criterion = nn.CrossEntropyLoss()

    def calc(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.criterion(y_pred, y_true)

    def __call__(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.calc(y_pred, y_true)

In [11]:
class Optimizer:
    def __init__(self, model: Model, lr: float = 0.01) -> None:
        self.optimizer = optim.AdamW(model.parameters(), lr)

    def step(self) -> None:
        self.optimizer.step()

    def zero_grad(self) -> None:
        self.optimizer.zero_grad()

In [12]:
def evaluation(model: Model, loss: Loss, data_loader: DataLoader, device: torch.device):
    model.eval()
    with torch.no_grad():
        total_nums = 0
        data_loss_value, data_acc_value = 0, 0
        for features, labels in data_loader:
            features, labels = features.to(device), labels.to(device)
            predict = model(features)
            loss_value = loss(predict, labels)

            total_nums += len(labels)
            data_loss_value += loss_value.item() * len(labels)
            data_acc_value += (predict.argmax(dim=1) == labels).sum().item()

        data_loss_value /= total_nums
        data_acc_value /= total_nums
    return data_loss_value, data_acc_value

In [13]:
def train(
    model: Model,
    loss: Loss,
    optimizer: Optimizer,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    nnum_epoches: int,
    device: torch.device,
):
    train_loss_list, train_acc_list = [], []
    valid_loss_list, valid_acc_list = [], []
    for epoch in range(nnum_epoches):
        model.train()
        total_nums = 0
        train_loss, train_acc = 0.0, 0.0
        valid_loss, valid_acc = 0.0, 0.0
        # 使用 tqdm 包裹 train_loader
        # desc: 设置进度条前面的文字
        # leave: 当前 Epoch 完成后是否保留进度条
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{nnum_epoches}", leave=True)
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            # 计算预测值和损失值并优化
            predict = model(images)
            loss_value = loss(predict, labels)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            # 统计数据
            total_nums += len(labels)
            train_loss += loss_value.item() * len(labels)
            train_acc += (predict.argmax(dim=1) == labels).sum().item()
            # 实时更新进度条右侧的显示信息
            pbar.set_postfix({"currentBatchLoss": f"{loss_value.item():.4f}"})
        # 计算整个 Epoch 的平均值
        train_loss /= total_nums
        train_acc /= total_nums
        valid_loss, valid_acc = evaluation(model, loss, valid_loader, device)
        # 3. 打印最终结果（在进度条完成后换行显示）
        print(
            f"epoch:{epoch+1}\r \
              TrainLoss: {train_loss:.4f}, TrainAcc: {train_acc:.4f}\r \
              ValidLoss: {valid_loss:.4f}, ValidAcc: {valid_acc:.4f}\r"
        )
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(valid_acc)
    return train_loss_list, train_acc_list, valid_loss_list, valid_acc_list

In [14]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    DATA_ROOT = (
        "D:/files/works/DeepLearning/experiment3/data/car-classificationproject-vision"
    )
    train_loader, val_loader, test_loader, num_classes = setup_data_loaders(
        data_root=DATA_ROOT,
        image_size=224,
        batch_size=2,
        split_ratio=0.8,
        num_workers=4,
    )
    model = Model(
        chns_in=3,
        chns_out=4,
        ker_size=3,
        nums_classes=45,
        device=device,
    ).to(device)
    loss = Loss()
    optimizer = Optimizer(model=model, lr=0.003)
    train_loss_list, train_acc_list, valid_loss_list, valid_acc_list = train(
        model=model,
        loss=loss,
        optimizer=optimizer,
        train_loader=train_loader,
        valid_loader=val_loader,
        nnum_epoches=2,
        device=device,
    )

检测到的类别数量: 45
训练集大小: 3240, 验证集大小: 810, 测试集大小: 450
--- 数据加载和划分完成 ---


Epoch 1/2:   0%|          | 1/1620 [07:35<204:38:49, 455.05s/it, currentBatchLoss=4.7537]


KeyboardInterrupt: 

## 2 使用torch.nn实现二维卷积，并在至少一个数据集上进行实验，从训练时间、预测精度、Loss变化等角度分析实验结果（最好使用图表展示）

In [ ]:
class Model(nn.Module):
    def __init__(
        self,
        chns_in: int,
        chns_base: int,
        feats_base: int,
        nums_classes: int,
        dropout_rate: float,
        ker_size: int,
        padding: int = 0,
        stride: int = 1,
    ) -> None:
        super().__init__()
        self.chns_in = chns_in
        self.chns_base = chns_base
        self.feats_base = feats_base
        self.nums_classes = nums_classes
        self.dropout_rate = dropout_rate
        self.ker_size = ker_size
        self.padding = padding
        self.stride = stride

        self.conv = nn.Sequential(
            # cov1
            nn.Conv2d(
                chns_in,
                chns_base * 1,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            # cov2
            nn.Conv2d(
                chns_base * 1,
                chns_base * 2,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            # conv3
            nn.Conv2d(
                chns_base * 2,
                chns_base * 4,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            # conv4
            nn.Conv2d(
                chns_base * 4,
                chns_base * 8,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 8),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        self.pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)))
        self.fc = nn.Sequential(
            # fc1
            nn.Linear(self.chns_base * 8, self.feats_base),
            # nn.BatchNorm1d(self.feats_base),
            nn.ReLU(inplace=True),
            nn.Dropout(p=self.dropout_rate),
            # # fc2
            # nn.Linear(self.feats_base, self.feats_base),
            # nn.BatchNorm1d(self.feats_base),
            # nn.ReLU(inplace=True),
            # nn.Dropout(p=self.dropout_rate),
            # fc3
            nn.Linear(self.feats_base, self.nums_classes),
        )
        # 权重初始化
        self._initialize_weights()

    def _initialize_weights(self):
        """Kaiming初始化"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.conv(X)
        X = self.pool(X)
        X = torch.flatten(X, 1)
        X = self.fc(X)
        return X

In [3]:
class Loss:
    def __init__(self) -> None:
        self.criterion = nn.CrossEntropyLoss()

    def calc(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.criterion(y_pred, y_true)

    def __call__(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.calc(y_pred, y_true)

In [4]:
class Optimizer:
    def __init__(
        self, model: Model, lr: float = 0.01, weight_decay: float = 0.01
    ) -> None:
        self.optimizer = optim.AdamW(model.parameters(), lr, weight_decay=weight_decay)

    def step(self) -> None:
        self.optimizer.step()

    def zero_grad(self) -> None:
        self.optimizer.zero_grad()

In [5]:
def evaluation(model: Model, loss: Loss, data_loader: DataLoader, device: torch.device):
    model.eval()
    with torch.no_grad():
        total_nums = 0
        data_loss_value, data_acc_value = 0, 0
        for features, labels in data_loader:
            features, labels = features.to(device), labels.to(device)
            predict = model(features)
            loss_value = loss(predict, labels)

            total_nums += len(labels)
            data_loss_value += loss_value.item() * len(labels)
            data_acc_value += (predict.argmax(dim=1) == labels).sum().item()

        data_loss_value /= total_nums
        data_acc_value /= total_nums
    return data_loss_value, data_acc_value

In [6]:
def train(
    model: Model,
    loss: Loss,
    optimizer: Optimizer,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    nnum_epoches: int,
    device: torch.device,
    accumulation_steps: int,  # 建议传入 4 或 8
):
    train_loss_list, train_acc_list = [], []
    valid_loss_list, valid_acc_list = [], []

    # 增加学习率调度器（建议配合梯度累加使用）
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    #     optimizer.optimizer, T_max=nnum_epoches
    # )

    for epoch in range(nnum_epoches):
        model.train()
        total_nums = 0
        train_loss, train_acc = 0.0, 0.0

        optimizer.zero_grad()  # 在循环开始前清零

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{nnum_epoches}", leave=True)
        for i, (images, labels) in enumerate(pbar):
            images, labels = images.to(device), labels.to(device)

            predict = model(images)
            raw_loss = loss(predict, labels)  # 保存原始 loss 用于统计

            # 1. 梯度缩放与反向传播
            loss_accumulated = raw_loss / accumulation_steps
            loss_accumulated.backward()

            # 2. 达到步数更新梯度
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            # 3. 统计数据使用原始 loss (raw_loss)
            total_nums += len(labels)
            train_loss += raw_loss.item() * len(labels)  # 这里用 raw_loss 保证统计准确
            train_acc += (predict.argmax(dim=1) == labels).sum().item()

            pbar.set_postfix({"batchLoss": f"{raw_loss.item():.4f}"})

        # 4. 处理 Epoch 末尾未达步数的残余梯度
        if len(train_loader) % accumulation_steps != 0:
            optimizer.step()
            optimizer.zero_grad()

        # 计算平均值
        train_loss /= total_nums
        train_acc /= total_nums

        # 5. 更新学习率
        # scheduler.step()

        valid_loss, valid_acc = evaluation(model, loss, valid_loader, device)

        print(
            f"epoch:{epoch+1}\n"
            f"TrainLoss: {train_loss:.4f}, TrainAcc: {train_acc:.4f}\n"
            f"ValidLoss: {valid_loss:.4f}, ValidAcc: {valid_acc:.4f}\n"
            f"LR: {optimizer.optimizer.param_groups[0]['lr']:.6f}"
        )  # 打印当前学习率辅助调试

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(valid_acc)

    return train_loss_list, train_acc_list, valid_loss_list, valid_acc_list

In [7]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    DATA_ROOT = (
        "D:/files/works/DeepLearning/experiment3/data/car-classificationproject-vision"
    )
    train_loader, val_loader, test_loader, num_classes = setup_data_loaders(
        data_root=DATA_ROOT,
        image_size=224,
        batch_size=32,
        split_ratio=0.8,
        num_workers=4,
    )
    model = Model(
        chns_in=3,
        chns_base=64,
        feats_base=1024,
        dropout_rate=0.3,
        ker_size=3,
        nums_classes=num_classes,
        padding=1,
        stride=1,
    ).to(device)
    loss = Loss()
    optimizer = Optimizer(model=model, lr=1e-3, weight_decay=1e-4)
    train_loss_list, train_acc_list, valid_loss_list, valid_acc_list = train(
        model=model,
        loss=loss,
        optimizer=optimizer,
        train_loader=train_loader,
        valid_loader=val_loader,
        nnum_epoches=64,
        device=device,
        accumulation_steps=8,
    )

✓ 检测到的类别数量: 45
✓ 训练集大小: 3240 (使用数据增强)
✓ 验证集大小: 810 (不使用数据增强)
✓ 测试集大小: 450
✓ 数据加载完成，Batch Size: 32


Epoch 1/64: 100%|██████████| 102/102 [00:29<00:00,  3.48it/s, batchLoss=3.7035]


epoch:1
TrainLoss: 3.7411, TrainAcc: 0.0528
ValidLoss: 3.6300, ValidAcc: 0.0593
LR: 0.001000


Epoch 2/64: 100%|██████████| 102/102 [00:34<00:00,  2.94it/s, batchLoss=3.7291]


epoch:2
TrainLoss: 3.5618, TrainAcc: 0.0858
ValidLoss: 3.4593, ValidAcc: 0.0975
LR: 0.001000


...

epoch:62
TrainLoss: 1.6670, TrainAcc: 0.5065
ValidLoss: 2.6907, ValidAcc: 0.3185
LR: 0.001000


Epoch 63/64: 100%|██████████| 102/102 [00:33<00:00,  3.00it/s, batchLoss=1.2201]


epoch:63
TrainLoss: 1.6560, TrainAcc: 0.5099
ValidLoss: 2.3302, ValidAcc: 0.3679
LR: 0.001000


Epoch 64/64: 100%|██████████| 102/102 [00:30<00:00,  3.39it/s, batchLoss=1.7620]


epoch:64
TrainLoss: 1.6107, TrainAcc: 0.5194
ValidLoss: 2.3748, ValidAcc: 0.3889
LR: 0.001000


## 3 不同超参数的对比分析（包括卷积层数、卷积核大小、batchsize、lr等）选其中至少1-2个进行分析

## 4 使用PyTorch实现经典模型AlexNet并在至少一个数据集进行试验分析 （无GPU环境则至少实现模型）

In [ ]:
class Model(nn.Module):
    def __init__(
        self,
        chns_in: int,
        chns_base: int,
        feats_base: int,
        nums_classes: int,
        ker_size: int,
        padding: int = 0,
        stride: int = 1,
    ) -> None:
        super().__init__()
        self.chns_in = chns_in
        self.chns_base = chns_base
        self.feats_base = feats_base
        self.nums_classes = nums_classes
        self.ker_size = ker_size
        self.padding = padding
        self.stride = stride

        self.conv = nn.Sequential(
            # cov1
            nn.Conv2d(
                chns_in,
                chns_base * 1,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            # cov2
            nn.Conv2d(
                chns_base * 1,
                chns_base * 2,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            # conv3
            nn.Conv2d(
                chns_base * 2,
                chns_base * 4,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 4),
            nn.ReLU(inplace=True),
            # conv4
            nn.Conv2d(
                chns_base * 4,
                chns_base * 8,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 8),
            nn.ReLU(inplace=True),
            # conv5
            nn.Conv2d(
                chns_base * 8,
                chns_base * 16,
                kernel_size=ker_size,
                padding=self.padding,
                stride=self.stride,
            ),
            nn.BatchNorm2d(chns_base * 16),
            nn.ReLU(inplace=True),
        )
        self.pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)))
        self.fc = nn.Sequential(
            nn.Linear(self.chns_base * 16, self.feats_base),
            nn.Linear(self.feats_base, self.nums_classes),
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.conv(X)
        X = self.pool(X)
        X = torch.flatten(X, 1)
        X = self.fc(X)
        return X

In [ ]:
class Loss:
    def __init__(self) -> None:
        self.criterion = nn.CrossEntropyLoss()

    def calc(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.criterion(y_pred, y_true)

    def __call__(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.calc(y_pred, y_true)

In [ ]:
class Optimizer:
    def __init__(self, model: Model, lr: float = 0.01) -> None:
        self.optimizer = optim.AdamW(model.parameters(), lr)

    def step(self) -> None:
        self.optimizer.step()

    def zero_grad(self) -> None:
        self.optimizer.zero_grad()

In [ ]:
def evaluation(model: Model, loss: Loss, data_loader: DataLoader, device: torch.device):
    model.eval()
    with torch.no_grad():
        total_nums = 0
        data_loss_value, data_acc_value = 0, 0
        for features, labels in data_loader:
            features, labels = features.to(device), labels.to(device)
            predict = model(features)
            loss_value = loss(predict, labels)

            total_nums += len(labels)
            data_loss_value += loss_value.item() * len(labels)
            data_acc_value += (predict.argmax(dim=1) == labels).sum().item()

        data_loss_value /= total_nums
        data_acc_value /= total_nums
    return data_loss_value, data_acc_value

In [ ]:
def train(
    model: Model,
    loss: Loss,
    optimizer: Optimizer,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    nnum_epoches: int,
    device: torch.device,
):
    train_loss_list, train_acc_list = [], []
    valid_loss_list, valid_acc_list = [], []
    for epoch in range(nnum_epoches):
        model.train()
        total_nums = 0
        train_loss, train_acc = 0.0, 0.0
        valid_loss, valid_acc = 0.0, 0.0
        # 使用 tqdm 包裹 train_loader
        # desc: 设置进度条前面的文字
        # leave: 当前 Epoch 完成后是否保留进度条
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{nnum_epoches}", leave=True)
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            # 计算预测值和损失值并优化
            predict = model(images)
            loss_value = loss(predict, labels)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            # 统计数据
            total_nums += len(labels)
            train_loss += loss_value.item() * len(labels)
            train_acc += (predict.argmax(dim=1) == labels).sum().item()
            # 实时更新进度条右侧的显示信息
            pbar.set_postfix({"currentBatchLoss": f"{loss_value.item():.4f}"})
        # 计算整个 Epoch 的平均值
        train_loss /= total_nums
        train_acc /= total_nums
        valid_loss, valid_acc = evaluation(model, loss, valid_loader, device)
        # 3. 打印最终结果（在进度条完成后换行显示）
        print(
            f"epoch:{epoch+1}\n\
              TrainLoss: {train_loss:.4f}, TrainAcc: {train_acc:.4f}\n\
              ValidLoss: {valid_loss:.4f}, ValidAcc: {valid_acc:.4f}\n"
        )
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(valid_acc)
    return train_loss_list, train_acc_list, valid_loss_list, valid_acc_list

In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    DATA_ROOT = (
        "D:/files/works/DeepLearning/experiment3/data/car-classificationproject-vision"
    )
    train_loader, val_loader, test_loader, num_classes = setup_data_loaders(
        data_root=DATA_ROOT,
        image_size=224,
        batch_size=24,
        split_ratio=0.8,
        num_workers=4,
    )
    model = Model(
        chns_in=3,
        chns_base=64,
        feats_base=1024,
        ker_size=3,
        nums_classes=45,
        padding=1,
        stride=1,
    ).to(device)
    loss = Loss()
    optimizer = Optimizer(model=model, lr=0.003)
    train_loss_list, train_acc_list, valid_loss_list, valid_acc_list = train(
        model=model,
        loss=loss,
        optimizer=optimizer,
        train_loader=train_loader,
        valid_loader=val_loader,
        nnum_epoches=64,
        device=device,
    )

## 5 使用实验2中的前馈神经网络模型在本次给定数据集上进行实验，并将实验结果与卷积模型结果进行对比分析（选做）